In [1]:
%run "data_random.ipynb"

In [2]:
ds_full = generate_dataset(60,10)

1 structure generated
2 structure generated
3 structure generated
4 structure generated
5 structure generated
6 structure generated
7 structure generated
8 structure generated
9 structure generated
10 structure generated


In [3]:
BLOCK_SIZE = 2
ds_red = generate_reduced_dataset(ds_full.dens_full.values, ds_full.bc_cond.values, ds_full.f_cond.values, factor=BLOCK_SIZE)
ds_block = generate_block_dataset(ds_full, ds_red, block_size=BLOCK_SIZE)

In [ ]:
dir = "Data"
ds_full.to_netcdf(dir+"/full.nc")
ds_block.to_netcdf(dir+"/block.nc")

In [4]:
BATCH_SIZE = 4
dens_full, disp_slice, outputs = extract_data(ds_full, ds_block)
dataset = tf.data.Dataset.from_tensor_slices(({"dens_input": dens_full, "disp_input": disp_slice}, outputs))
train_dataset, _, test_dataset = get_dataset(dataset, dens_full.shape[0], batch_size=BATCH_SIZE)

num_of_layers_cnn, num_of_layers_fnn =10, 10
cnn = cnn_model(num_of_layers=num_of_layers_cnn)
fnn = fnn_model(block_size=BLOCK_SIZE, num_of_layers=num_of_layers_fnn)

In [5]:
test_losses, val_losses = custom_train(block_size=BLOCK_SIZE,cnn_model=cnn,fnn_model=fnn,\
                        loss_function=tf.keras.losses.MeanAbsoluteError(),\
                        num_epoch=200,optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),\
                        batched_train=train_dataset, batched_test=test_dataset, num_of_layers_1=num_of_layers_cnn, num_of_layers_2=num_of_layers_fnn,save_model=False, 
                        casename='test')

: 

: 